# Using EIGN's Laplacian and Convolution Operators

Here, we will explore in further detail how to use the Laplacian operators and the associated convolutions to build your own model with EIGN's layers.

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

### Create a dummy graph

Let's create a dummy graph with some directed and undirected edges. We assign each edge with features that have an inherent direction (signed or orientation equivariant features) and some features that do not have an inherent direction (unsigned or orientation invariant features).

The signed features are represented with respect to some arbitrary edge orientation through their sign. The edge orientation is encoded through the `edge_index`: If it contains an edge `(u, v)`, signed features are represented relative to the orientation `u -> v`.


In [2]:
num_signed_features = 3
num_unsigned_features = 6

# Represent the graph
edge_index = (
    torch.tensor(
        [
            [0, 1],
            [1, 2],
            [2, 3],
            [2, 4],
            [3, 5],
            [5, 0],
            [5, 2],
        ]
    )
    .t()
    .contiguous()
)
edge_is_directed = torch.tensor([0, 0, 0, 0, 0, 1, 1], dtype=torch.bool)

dummy_features_signed = torch.randn(edge_index.size(1), num_signed_features)
dummy_features_unsigned = torch.randn(edge_index.size(1), num_unsigned_features)

### The Magnetic Edge Laplacian

Our code materializes the magnetic operators at the core of EIGN using sparse matrices to scale to large graphs. By introducing a phase shift, these operators are complex-valued.

In [3]:
from eign import magnetic_edge_laplacian, magnetic_incidence_matrix

In [4]:
L_signed_to_signed = magnetic_edge_laplacian(
    edge_index,
    edge_is_directed,
    q=1 / edge_index.size(1),
    signed_in=True,
    signed_out=True,
)
L_signed_to_unsigned = magnetic_edge_laplacian(
    edge_index,
    edge_is_directed,
    q=1 / edge_index.size(1),
    signed_in=True,
    signed_out=False,
)
L_unsigned_to_signed = magnetic_edge_laplacian(
    edge_index,
    edge_is_directed,
    q=1 / edge_index.size(1),
    signed_in=False,
    signed_out=True,
)
L_unsigned_to_unsigned = magnetic_edge_laplacian(
    edge_index,
    edge_is_directed,
    q=1 / edge_index.size(1),
    signed_in=False,
    signed_out=False,
)
L_signed_to_signed

/nfs/homedirs/fuchsgru/magnetic_edge_gnn/eign/src/eign/laplacian.py:112: UserWarning: Sparse CSR tensor support is in beta state. If you miss a functionality in the sparse tensor support, please submit a feature request to https://github.com/pytorch/pytorch/issues. (Triggered internally at ../aten/src/ATen/SparseCsrTensorImpl.cpp:53.)
  laplacian = (


tensor(indices=tensor([[0, 0, 0, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 3, 3, 3, 3, 4, 4,
                        4, 4, 5, 5, 5, 5, 6, 6, 6, 6, 6, 6],
                       [0, 1, 5, 0, 1, 2, 3, 6, 1, 2, 3, 4, 6, 1, 2, 3, 6, 2, 4,
                        5, 6, 0, 4, 5, 6, 1, 2, 3, 4, 5, 6]]),
       values=tensor([ 2.0000+0.0000j, -1.0000+0.0000j, -0.9010+0.4339j,
                      -1.0000+0.0000j,  2.0000+0.0000j, -1.0000+0.0000j,
                      -1.0000+0.0000j,  0.9010-0.4339j, -1.0000+0.0000j,
                       2.0000+0.0000j,  1.0000+0.0000j, -1.0000+0.0000j,
                      -0.9010+0.4339j, -1.0000+0.0000j,  1.0000+0.0000j,
                       2.0000+0.0000j, -0.9010+0.4339j, -1.0000+0.0000j,
                       2.0000+0.0000j, -0.9010-0.4339j, -0.9010-0.4339j,
                      -0.9010-0.4339j, -0.9010+0.4339j,  2.0000+0.0000j,
                       1.0000+0.0000j,  0.9010+0.4339j, -0.9010-0.4339j,
                      -0.9010-0.4339j, -0.9010+0.4339j,  

These operators can be used as (orientation equivariant / invariant) graph shift operator for edge signals

In [5]:
L_signed_to_signed @ dummy_features_signed.to(L_signed_to_signed.dtype)
L_signed_to_unsigned @ dummy_features_signed.to(L_signed_to_unsigned.dtype)
L_unsigned_to_unsigned @ dummy_features_unsigned.to(L_unsigned_to_unsigned.dtype)
L_unsigned_to_signed @ dummy_features_unsigned.to(L_unsigned_to_signed.dtype)

tensor([[-1.3976-0.1484j,  2.0847-0.8242j, -1.3104+0.1457j,  0.0156-0.0762j,
          0.4481-0.0593j,  0.2663+0.2147j],
        [ 3.0569-0.4861j,  1.5729-0.1470j,  2.6326-0.7833j, -4.6981-0.1897j,
         -0.6818-0.0484j, -1.0065-0.2472j],
        [ 1.0951+0.4861j, -0.3666+0.1470j, -0.3548+0.7833j,  4.1013+0.1897j,
          0.8220+0.0484j, -0.7784+0.2472j],
        [-0.8543+0.4861j,  0.4231+0.1470j, -1.2198+0.7833j,  1.3796+0.1897j,
         -1.4880+0.0484j, -1.2642+0.2472j],
        [-0.8495+0.3377j, -0.3049-0.6773j,  1.3282+0.9291j,  1.8665+0.1135j,
         -1.0853-0.0109j,  0.9206+0.4619j],
        [-1.9271-1.1252j, -2.7543-0.2084j, -2.4082-0.0667j, -1.2403+1.0477j,
          0.4268+1.0513j,  0.4315+0.2827j],
        [-0.1568-0.9769j,  0.9626+0.5035j, -0.9403-0.0675j, -5.3437-1.2234j,
          0.2630+0.8529j, -0.2212-0.0677j]])

Be mindful that for `q != 0`, these shift operators are complex-valued and so must their inputs. A convenient way to combine this with learnable feature transformations that operate on real values is to flatten and unflatten the edge signal. Effectively, we therefore model the real and complex part of the signals.

In [6]:
num_signed_features_out = 16
lin = nn.Linear(num_signed_features, num_signed_features_out)

hidden_signed_complex = torch.view_as_complex(
    lin(dummy_features_signed).reshape(-1, num_signed_features_out // 2, 2)
)
hidden_signed_complex = L_signed_to_signed @ hidden_signed_complex
hidden_signed = torch.view_as_real(hidden_signed_complex).reshape(
    -1, num_signed_features_out
)
hidden_signed.size()

torch.Size([7, 16])

### Node-level signals

The magnetic incidence matrices define mappings from edge signals to node signals and vice versa. This can be used to incorporate or predict node level outputs while still satisfying orientation equivariance or invariance.

These node level signals are invariant to orientation changes of *undirected* edges. Therefore, any transformation (that does not depend on orientation) can be used to transform them, if desired (see `eign_example.ipynb`).

In [7]:
B_signed = magnetic_incidence_matrix(
    edge_index, edge_is_directed, q=1 / edge_index.size(1), signed=True
)

In [8]:
node_level = B_signed @ hidden_signed.to(B_signed.dtype)
node_level.size()

torch.Size([6, 16])

Alternatively, the boundary operator can transform node level signals to the edge level.

In [9]:
node_level = torch.randn(
    int(edge_index.max().item()) + 1, num_signed_features_out, dtype=B_signed.dtype
)
edge_level = B_signed.conj().t() @ node_level
edge_level.size()

torch.Size([7, 16])

### Built-in Convolution operators

The `eign` package also provides convolution operators that use these Laplacians. Each of these convolutions uses a single linear layer as an edge-feature transformation.

In [10]:
from eign import (
    MagneticEdgeLaplacianConv,
    MagneticEdgeLaplacianWithNodeTransformationConv,
)

In [11]:
conv = MagneticEdgeLaplacianConv(
    num_signed_features,
    num_signed_features_out,
    q=1 / edge_index.size(1),
    signed_in=True,
    signed_out=True,
    bias=None,  # Setting `bias` to `None` automatically uses a bias only if orientation equivariance / invariance is preserved
).eval()
conv

MagneticEdgeLaplacianConv(
  (lin): Linear(in_features=3, out_features=16, bias=False)
)

In [12]:
conv(dummy_features_signed, edge_index, edge_is_directed)

tensor([[ 2.2376e-01,  2.3321e-01,  1.0154e-01,  4.4024e-01, -8.1171e-02,
         -5.0108e-02,  1.6376e-01,  1.1127e-01,  4.1456e-01, -1.5332e-01,
         -1.7078e-01,  9.4374e-02,  4.7508e-01,  2.7954e-01, -1.7467e-01,
          3.1720e-01],
        [-5.2929e-01, -5.4653e-01, -6.4605e-02, -5.0673e-01,  3.0910e-01,
          1.0555e-01, -7.2847e-03, -2.9625e-01, -5.4794e-01,  3.0851e-01,
          1.6006e-01, -9.4288e-02, -5.3793e-01, -4.2596e-01,  2.0172e-01,
         -5.6020e-01],
        [ 7.0587e-01,  6.9223e-01, -6.7022e-04,  4.0147e-01, -5.0969e-01,
         -8.8372e-02,  8.3006e-02,  4.8430e-01,  4.0839e-01, -4.2072e-01,
         -2.4215e-01,  1.7607e-01,  4.5952e-01,  4.1288e-01, -1.1597e-01,
          5.6947e-01],
        [ 7.5829e-01,  6.3479e-01,  8.3270e-02,  5.4759e-01, -4.3006e-01,
         -8.5693e-02, -5.1941e-02,  3.8820e-01,  5.8925e-01, -4.2550e-01,
         -1.2657e-01,  9.7673e-02,  6.1236e-01,  4.6654e-01, -1.6139e-01,
          7.0615e-01],
        [-3.7523e-01

You can also define your own node-level feature transformation. For example, you could inform the convolution about node features that are available.

In [13]:
class MyNodeTransformation(nn.Module):
    def __init__(
        self, num_features_in_edge, num_features_out_edge, num_features_in_node
    ):
        super().__init__()
        self.edge_level = nn.Linear(num_features_in_edge, num_features_out_edge)
        self.node_level = nn.Linear(num_features_in_node, num_features_out_edge)

    def forward(self, edge_level, node_level):
        return F.relu(self.edge_level(edge_level) + self.node_level(node_level))


def initialize_node_feature_transformation(
    in_channels: int, out_channels: int, num_features_in_node
) -> nn.Module:
    return MyNodeTransformation(in_channels, out_channels, num_features_in_node)


conv = MagneticEdgeLaplacianWithNodeTransformationConv(
    num_signed_features,
    num_signed_features_out,
    q=1 / edge_index.size(1),
    signed_in=True,
    signed_out=True,
    initialize_node_feature_transformation=initialize_node_feature_transformation,
    # Additional args and kwargs are passed to `initialize_node_feature_transformation`
    num_features_in_node=13,
).eval()

node_level_signal = torch.randn(int(edge_index.max().item()) + 1, 13)
conv(
    dummy_features_signed,
    edge_index,
    edge_is_directed,
    # Additional args and kwargs are passed to `initialize_node_feature_transformation.forward`
    node_level_signal,
)

tensor([[-0.4800, -0.0344,  0.0086,  0.1173,  0.3124,  0.2384, -0.3932,  0.2863,
          0.0809, -0.4113,  0.0000,  0.1012, -0.4231,  0.0000, -0.1464, -0.6657],
        [ 0.3667,  0.2890, -0.0189, -0.0958, -0.2551, -0.1947,  0.6252, -0.2338,
         -0.0660,  0.1818,  0.3318,  0.1568,  0.0000,  0.0830,  0.0000,  0.5429],
        [-0.2936, -0.1894,  0.0810,  0.0000,  0.0000,  0.0000, -0.6035,  0.0000,
          0.1187, -0.1818, -0.2843, -0.2678,  0.1181, -0.0830,  0.0024, -0.3616],
        [-0.2452, -0.2099,  0.0000,  0.3732,  0.5058,  0.1298, -0.5782,  0.0042,
          0.3029,  0.0266, -0.3635,  0.2527,  0.0000, -0.0910,  0.2457,  0.0759],
        [-0.1669,  0.1031,  0.1791,  0.3422,  0.0288,  0.0000,  0.2200,  0.0000,
         -0.1300,  0.0000, -0.0521, -0.0124,  0.2042,  0.2851,  0.0960, -0.1985],
        [ 0.3521,  0.0604, -0.3781, -0.1864, -0.0260,  0.0125,  0.0972,  0.2583,
         -0.1596,  0.3314, -0.0188,  0.0391, -0.0833,  0.0521, -0.2292,  0.6361],
        [ 0.1717,  0.2

### Using EIGN Blocks

One block of EIGN uses convolutions and a fusion layer to model local interactions. This is also encapsulated in a block class that can be used if so desired.

In [14]:
from eign import (
    EIGNBlockMagneticEdgeLaplacianConv,
    EIGNBlockMagneticEdgeLaplacianWithNodeTransformationConv,
)

In [15]:
num_unsigned_features_out = 16

In [16]:
block = EIGNBlockMagneticEdgeLaplacianConv(
    num_signed_features,
    num_signed_features_out,
    num_unsigned_features,
    num_unsigned_features_out,
    q=1 / edge_index.size(1),
).eval()
block

EIGNBlockMagneticEdgeLaplacianConv(
  (signed_fusion_layer): FusionLayer(
    (lin_layer1): Linear(in_features=16, out_features=16, bias=False)
    (lin_layer2): Linear(in_features=16, out_features=16, bias=False)
  )
  (unsigned_fusion_layer): FusionLayer(
    (lin_layer1): Linear(in_features=16, out_features=16, bias=True)
    (lin_layer2): Linear(in_features=16, out_features=16, bias=True)
  )
  (unsigned_conv): ResidualWrapper(
    (lin): Linear(in_features=6, out_features=16, bias=False)
    (conv): MagneticEdgeLaplacianConv(
      (lin): Linear(in_features=6, out_features=16, bias=False)
    )
  )
  (unsigned_signed_conv): MagneticEdgeLaplacianConv(
    (lin): Linear(in_features=6, out_features=16, bias=False)
  )
  (signed_conv): ResidualWrapper(
    (lin): Linear(in_features=3, out_features=16, bias=False)
    (conv): MagneticEdgeLaplacianConv(
      (lin): Linear(in_features=3, out_features=16, bias=False)
    )
  )
  (signed_unsigned_conv): MagneticEdgeLaplacianConv(
    (lin

In [17]:
output_signed, output_unsigned = block(
    dummy_features_signed, dummy_features_unsigned, edge_index, edge_is_directed
)

In [18]:
output_signed

tensor([[ 0.3581,  0.2057,  0.4567, -0.3503,  0.5559, -0.1147,  0.3204,  0.1869,
         -0.3732, -0.1508, -0.6672, -0.4099,  0.8753, -0.4462, -0.2957, -0.5566],
        [-0.2667,  0.5258, -0.3243, -0.5295, -0.2900, -0.7422, -0.6942,  0.1552,
         -0.6747, -0.0857,  0.3551, -0.9293,  0.6151, -0.8680,  0.5882, -0.4349],
        [ 0.2575,  0.1130, -0.3635,  0.4356, -0.8739,  0.1601, -0.1613, -0.4904,
         -0.1907, -0.6528,  0.6595,  0.2372,  0.2861,  0.8399,  0.0780,  0.4730],
        [ 0.6064, -0.9089,  0.4926,  0.3841,  0.6003,  1.0251,  0.9099, -0.3473,
          0.8668,  0.6405, -0.5512,  0.8687, -0.8110,  0.7201, -0.6522,  0.4163],
        [-0.7042,  0.9108, -0.7157,  0.4774, -0.9342, -0.1898, -0.8767, -0.7547,
         -0.8977, -0.4987,  0.9212, -0.3779,  0.8107, -0.4205,  0.4524, -0.5124],
        [ 0.7462,  0.6828, -0.0820, -0.8530, -0.4785, -0.4315, -0.6103,  0.4677,
         -0.7656, -0.7704, -0.5445, -0.8427,  0.8008, -0.5492,  0.3564, -0.3676],
        [-0.1096, -0.4

In [19]:
output_unsigned

tensor([[ 2.6522e-01,  4.0882e-01,  1.0961e+00,  2.7800e-01, -3.1668e-02,
          3.1696e-01, -2.2765e-03,  2.8104e-01,  1.4925e+00,  7.5996e-01,
         -7.7084e-02,  1.0134e+00,  7.2478e-01,  3.6854e-02,  4.3876e-02,
         -1.0827e-02],
        [-2.8249e-02, -1.2893e-02,  1.2853e+00,  6.9410e-01,  4.9405e-04,
          1.3311e-02,  2.4489e-02, -1.5311e-01,  7.5778e-01,  1.1511e+00,
         -1.4159e-01,  2.0968e-01,  4.2797e-01,  2.2610e-02,  1.5517e-01,
         -1.6206e-03],
        [-1.5499e-01,  3.4916e-01, -1.0239e-02,  1.5702e+00,  2.0533e-02,
          7.7504e-01,  4.3553e-02, -3.4818e-01,  1.3345e+00,  1.0472e+00,
          3.4267e-01,  1.1030e+00,  8.0169e-03,  3.4230e-02,  9.6021e-02,
          5.4491e-01],
        [-2.0536e-01, -1.0484e-02,  6.2879e-01,  8.3553e-01, -1.4406e-02,
          1.1463e+00,  2.4830e-01, -7.5914e-01,  7.5223e-01,  1.5769e+00,
         -7.3916e-02,  2.4655e-01,  5.7534e-04,  3.4806e-01,  2.1674e-01,
          5.8474e-01],
        [ 2.3003e-01

Again, you can also learn node feature representations induced by the edge signals.

In [20]:
block = EIGNBlockMagneticEdgeLaplacianWithNodeTransformationConv(
    num_signed_features,
    num_signed_features_out,
    num_unsigned_features,
    num_unsigned_features_out,
    q=1 / edge_index.size(1),
    initialize_node_feature_transformation=lambda num_in, num_out: nn.Sequential(
        nn.ReLU(), nn.Linear(num_in, num_out)
    ),
).eval()
block

EIGNBlockMagneticEdgeLaplacianWithNodeTransformationConv(
  (signed_fusion_layer): FusionLayer(
    (lin_layer1): Linear(in_features=16, out_features=16, bias=False)
    (lin_layer2): Linear(in_features=16, out_features=16, bias=False)
  )
  (unsigned_fusion_layer): FusionLayer(
    (lin_layer1): Linear(in_features=16, out_features=16, bias=True)
    (lin_layer2): Linear(in_features=16, out_features=16, bias=True)
  )
  (unsigned_conv): ResidualWrapper(
    (lin): Linear(in_features=6, out_features=16, bias=False)
    (conv): MagneticEdgeLaplacianWithNodeTransformationConv(
      (lin): Linear(in_features=6, out_features=16, bias=False)
      (node_feature_transformation): Sequential(
        (0): ReLU()
        (1): Linear(in_features=16, out_features=16, bias=True)
      )
    )
  )
  (unsigned_signed_conv): MagneticEdgeLaplacianWithNodeTransformationConv(
    (lin): Linear(in_features=6, out_features=16, bias=False)
    (node_feature_transformation): Sequential(
      (0): ReLU()
  

In [21]:
output_signed, output_unsigned = block(
    dummy_features_signed, dummy_features_unsigned, edge_index, edge_is_directed
)